In [22]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
import re
import time
import math
from alive_progress import alive_bar

# NYT API key
keyNYT = "BwemetlgrQFLmyML7BqSZqCjYgMvyANE"


In [19]:
# Directories
ROOT_DIR = os.path.dirname(os.path.abspath("__file__"))
PARENT_DIR = os.path.dirname(ROOT_DIR)

# Query setup function
def NYT(page):
    return requests.get("https://api.nytimes.com/svc/search/v2/articlesearch.json?&api-key=" + keyNYT +
  "&begin_date=20220201" +
  "&sort=newest" +
  "&fl=web_url%2Cheadline%2Cword_count%2Csnippet%2Clead_paragraph%2Cpub_date%2Ctype_of_material%2Cdocument_type" + 
  "&fq=type_of_material%3A(%22News%22)&page=" + str(page) +
  "&q=ukraine")

def numArticlesInPage(json):
    if json["response"]["meta"]["hits"] - json["response"]["meta"]["offset"] >= 10:
        return 10
    else:
        return json["response"]["meta"]["hits"] - json["response"]["meta"]["offset"] + 1

# Instancing a query to fetch basic information
json_NYT = NYT(1).json()
#numPages = json_guardian["response"]["headerspages"]
numArticles = json_NYT["response"]["meta"]["hits"]
numPages = ceil(numArticles/10)
#print(json.dumps(json_NYT, indent=2))

print(f"-> There are {numArticles} articles in {numPages} pages for the period and tags specified.")

-> There are 3065 articles in 307 pages for the period and tags specified.


In [ ]:
print(json.dumps(json_NYT, indent=2))

In [ ]:
# Instancing
urls = []
titles = []
bodies = []
dates = []
snippets = []


# Scraper
with alive_bar(numArticles, title="-> API Query", spinner="dots_waves", bar="smooth", force_tty=True) as bar:

    # Going through all pages available for the query
    for i in range(1,numPages):

        time.sleep(6) #NYT request limit
        json_NYT = NYT(i).json()

        # Going through all articles in a page
        for j in range(0,numArticlesInPage(json_NYT)):
            
            urls.append(json_NYT["response"]["docs"][j]["web_url"])
            dates.append(json_NYT["response"]["docs"][j]["pub_date"])

            titles.append(json_NYT["response"]["docs"][j]["headline"]["main"])
            #titles.append(re.sub(r"\|.*$", "", title)) # removing authors from titles

            body = BeautifulSoup(json_NYT["response"]["docs"][j]["lead_paragraph"], "html.parser").get_text()
            #body = replace_all(body, rep)  # replacing substrings
            bodies.append(re.sub(r"[\t\r\n]", "", body)) # removing line breaks

            snippet = BeautifulSoup(json_NYT["response"]["docs"][j]["snippet"], "html.parser").get_text() 
            snippets.append(re.sub(r"^[^—]+—\s*", "", snippet)) 
            bar()

# Transforming fetched info to dataframe
data = pd.DataFrame({"URL": urls, "Date": dates, "Title": titles, "Text": bodies, "Snippet" : snippets})

# Saving to csv
os.makedirs(PARENT_DIR + "/data", exist_ok=True)
data.to_csv(PARENT_DIR + "/data/NYT.csv", index=True, header=True)

print(f"-> {len(data)} articles fetched successfully!")


In [24]:
data.head(25)

,URL,Date,Title,Text,Snippet
0,https://www.nytimes.com/2022/06/20/us/politics...,2022-06-20T21:41:22+0000,Biden Says He Is Considering Seeking a Gas Tax...,"REHOBOTH BEACH, Del. — President Biden said on...","With regular gasoline around $5 per gallon, su..."
1,https://www.nytimes.com/2022/06/20/world/europ...,2022-06-20T21:17:34+0000,The Kremlin says two captured American fighter...,The Kremlin’s chief spokesman told NBC News on...,Moscow says the Americans who fought for Ukrai...
2,https://www.nytimes.com/2022/06/20/world/europ...,2022-06-20T18:57:52+0000,Russia’s blockade of Ukraine’s food exports is...,The Black Sea blockade that is preventing Ukra...,The war is contributing to global food shortag...
3,https://www.nytimes.com/2022/06/20/business/du...,2022-06-20T18:37:56+0000,Dutch Government Activates ‘Early Warning’ Bec...,The Dutch government said on Monday that Russi...,The government will allow more burning of coal...
4,https://www.nytimes.com/2022/06/20/world/europ...,2022-06-20T17:27:23+0000,"Russia steps up bombardments of Kharkiv, where...","KYIV, Ukraine — Shelling in and around Ukraine...",
5,https://www.nytimes.com/2022/06/20/world/europ...,2022-06-20T16:23:01+0000,Hear the stories of Ukrainians who were forcib...,"Since Russia invaded Ukraine in February, thou...",
6,https://www.nytimes.com/2022/06/20/world/russi...,2022-06-20T11:48:10+0000,Russia becomes China’s biggest crude-oil suppl...,Russia surpassed Saudi Arabia to become China’...,
7,https://www.nytimes.com/2022/06/20/world/europ...,2022-06-20T10:25:51+0000,A Fragmented Parliament Brings Macron Back Dow...,PARIS — President Emmanuel Macron earned the n...,President Emmanuel Macron largely sidelined Pa...
8,https://www.nytimes.com/2022/06/20/opinion/rec...,2022-06-20T09:00:22+0000,Washington Might Be About to Do Something Righ...,NASHVILLE — The media hasn’t paid much attenti...,An “unthinkable level of bipartisan support” m...
9,https://www.nytimes.com/2022/06/20/world/asia/...,2022-06-20T04:01:17+0000,New Brand of Activist Takes Aim at Ukraine War...,"BRUSSELS — Emmanuel Macron, the president of F...",to end the fighting and to save the planet.
